In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import LLMChain
import os

In [3]:
conversation = """
Financial Advisor: Good morning! How can I assist you today?
Customer: Hi, I'm looking for some advice on retirement planning. I'm 45 years old and want to ensure I have enough saved up.
Financial Advisor: Sure, I'd be happy to help with that. Do you currently have any retirement accounts, such as a 401(k) or an IRA?
Customer: Yes, I have a 401(k) through my employer and a Roth IRA that I contribute to regularly.
Financial Advisor: Great. How much do you currently have saved in those accounts?
Customer: I have about $150,000 in my 401(k) and $50,000 in my Roth IRA.
Financial Advisor: That's a good start. Are you currently making the maximum contributions to both accounts?
Customer: I'm contributing the maximum to my Roth IRA, but I'm only contributing 5% of my salary to my 401(k).
Financial Advisor: Increasing your 401(k) contributions could be beneficial, especially if your employer offers matching contributions. Do they match your 401(k) contributions?
Customer: Yes, they match 50% of my contributions up to 6% of my salary.
Financial Advisor: In that case, you might consider increasing your contribution to at least 6% to take full advantage of the employer match. It’s essentially free money for your retirement.
Customer: That makes sense. I'll look into increasing my contribution. What about other investment options? Should I diversify more?
Financial Advisor: Diversification is definitely important. Are you invested in a mix of stocks, bonds, and other assets within your 401(k) and Roth IRA?
Customer: I think so, but I'm not sure about the exact allocation.
Financial Advisor: We can review your current portfolio allocation. Generally, a mix of 60% stocks and 40% bonds is recommended for someone your age, but it depends on your risk tolerance. Would you say you are conservative, moderate, or aggressive in your investment approach?
Customer: I’d say I'm moderate. I want growth but also want to minimize risk.
Financial Advisor: Based on that, a 60/40 or even a 70/30 stock-to-bond ratio might be suitable. Also, consider including some international stocks and real estate investments to further diversify your portfolio.
Customer: That sounds good. Is there anything else I should be doing to prepare for retirement?
Financial Advisor: Besides saving and investing wisely, it's also important to manage your debt, keep an emergency fund, and regularly review and adjust your retirement plan. Do you have an emergency fund in place?
Customer: Yes, I have about six months' worth of living expenses saved in a high-yield savings account.
Financial Advisor: That’s excellent. You're on the right track. Regularly reviewing your retirement plan and making adjustments as needed is key to staying on course.
Customer: Thank you for the advice. I feel more confident about my retirement planning now.
Financial Advisor: You’re welcome! If you have any other questions or need further assistance, feel free to reach out.
"""

In [5]:
# Authenticating OpenAI services

os.environ['OPENAI_API_KEY'] = "sk-proj-n40SdhtnchmUMUXn64wwT3BlbkFJICSq4aqyi4sKHClnW2O5"

In [23]:
chat_model = ChatOpenAI(temperature=0,
                        model = 'gpt-4o')

In [24]:
# Function for summarization

def conversation_Summary(conversation):
    summary_prompt = "Please summarize the following conversation: \n{conversation}"
    summary_template = ChatPromptTemplate.from_template(summary_prompt)
    summary_chain = LLMChain(llm=chat_model,
                             prompt = summary_template,
                             output_key = 'summary')
    result =  summary_chain.run(conversation)
    return result

In [25]:
# function for Key point extraction

def conversation_Keypoints(conversation):
    keypoints_prompt = "Please extract the key information spoken in the following conversation: \n{conversation}"
    keypoints_template = ChatPromptTemplate.from_template(keypoints_prompt)
    keypoints_chains = LLMChain(llm=chat_model, 
                                prompt=keypoints_template,
                                output_key = 'keypoints')
    result = keypoints_chains.run(conversation)
    return result

In [16]:
print(conversation_Summary(conversation=conversation))

The financial advisor assists a 45-year-old customer with retirement planning. The customer currently has a 401(k) and a Roth IRA, with $150,000 and $50,000 saved, respectively. The advisor suggests increasing 401(k) contributions to take advantage of employer matching, recommends a 60/40 or 70/30 stock-to-bond ratio, and advises diversifying with international stocks and real estate investments. The customer is advised to manage debt, maintain an emergency fund, and regularly review and adjust their retirement plan. The customer feels more confident about their retirement planning after the conversation.


In [18]:
print(conversation_Keypoints(conversation=conversation))

- Customer is 45 years old and looking for advice on retirement planning
- Customer has a 401(k) with $150,000 and a Roth IRA with $50,000
- Customer is contributing the maximum to Roth IRA but only 5% of salary to 401(k)
- Employer matches 50% of contributions up to 6% of salary
- Financial Advisor recommends increasing 401(k) contribution to take full advantage of employer match
- Financial Advisor recommends diversifying investment portfolio with a mix of stocks, bonds, international stocks, and real estate
- Customer has six months' worth of living expenses saved in a high-yield savings account
- Financial Advisor emphasizes the importance of managing debt, having an emergency fund, and regularly reviewing and adjusting retirement plan.


In [26]:
print(conversation_Summary(conversation=conversation))

In the conversation, a 45-year-old customer seeks retirement planning advice from a financial advisor. The customer has $150,000 in a 401(k) and $50,000 in a Roth IRA, contributing the maximum to the Roth IRA but only 5% of their salary to the 401(k). The advisor suggests increasing the 401(k) contribution to at least 6% to maximize employer matching. They discuss the importance of diversification, recommending a 60/40 or 70/30 stock-to-bond ratio based on the customer's moderate risk tolerance, and suggest including international stocks and real estate. The advisor also emphasizes managing debt, maintaining an emergency fund, and regularly reviewing the retirement plan. The customer has an emergency fund with six months' living expenses and feels more confident after the discussion.


In [27]:
print(conversation_Keypoints(conversation=conversation))

Here is the key information extracted from the conversation:

1. **Customer's Age and Goal**:
   - Customer is 45 years old and seeking retirement planning advice.

2. **Current Retirement Accounts**:
   - Customer has a 401(k) with $150,000.
   - Customer has a Roth IRA with $50,000.

3. **Contributions**:
   - Customer is contributing the maximum to the Roth IRA.
   - Customer is contributing 5% of their salary to the 401(k).

4. **Employer Matching**:
   - Employer matches 50% of 401(k) contributions up to 6% of the salary.

5. **Advisor's Recommendations**:
   - Increase 401(k) contributions to at least 6% to take full advantage of the employer match.
   - Diversify investments within the 401(k) and Roth IRA.
   - Consider a mix of 60% stocks and 40% bonds, or 70/30, based on moderate risk tolerance.
   - Include international stocks and real estate investments for further diversification.

6. **Additional Financial Advice**:
   - Manage debt.
   - Maintain an emergency fund.
   - 

In [28]:
def verify_key_points(conversation, key_points):
    # Split key points into lines
    key_points_lines = key_points.strip().split('\n')
    # Initialize a dictionary to hold the verification results
    verification_results = {}
    
    for key_point in key_points_lines:
        # Check if each key point is in the conversation
        verification_results[key_point] = key_point.strip() in conversation

    return verification_results

In [30]:
key_points = conversation_Keypoints(conversation=conversation)

In [31]:
# Verify the extracted key points
verification_results = verify_key_points(conversation, key_points)
print("\nVerification Results:")
for key_point, is_present in verification_results.items():
    print(f"{key_point}: {'Present' if is_present else 'Not Present'}")


Verification Results:
Here is the key information from the conversation:: Not Present
: Present
- **Customer's Age**: 45 years old.: Not Present
- **Current Retirement Accounts**: : Not Present
  - 401(k) with $150,000 saved.: Not Present
  - Roth IRA with $50,000 saved.: Not Present
- **Contributions**:: Not Present
  - Maximum contributions to Roth IRA.: Not Present
  - 5% of salary to 401(k).: Not Present
- **Employer 401(k) Match**: 50% of contributions up to 6% of salary.: Not Present
- **Advice Given**:: Not Present
  - Increase 401(k) contributions to at least 6% to take full advantage of the employer match.: Not Present
  - Diversify investments within 401(k) and Roth IRA.: Not Present
  - Recommended portfolio allocation: 60% stocks and 40% bonds, or 70% stocks and 30% bonds, based on moderate risk tolerance.: Not Present
  - Consider including international stocks and real estate investments.: Not Present
- **Additional Recommendations**:: Not Present
  - Manage debt.: Not P

## Verification part

In [33]:
text_splitter =  RecursiveCharacterTextSplitter(chunk_size = 500,
                                                chunk_overlap = 40,
                                                separators=["\n\n", "\n", " ", ""])

In [36]:
splits =  text_splitter.split_text(conversation)

In [39]:
print(len(conversation))
print(len(splits))

2950
7


In [40]:
splits

["Financial Advisor: Good morning! How can I assist you today?\nCustomer: Hi, I'm looking for some advice on retirement planning. I'm 45 years old and want to ensure I have enough saved up.\nFinancial Advisor: Sure, I'd be happy to help with that. Do you currently have any retirement accounts, such as a 401(k) or an IRA?\nCustomer: Yes, I have a 401(k) through my employer and a Roth IRA that I contribute to regularly.\nFinancial Advisor: Great. How much do you currently have saved in those accounts?",
 "Customer: I have about $150,000 in my 401(k) and $50,000 in my Roth IRA.\nFinancial Advisor: That's a good start. Are you currently making the maximum contributions to both accounts?\nCustomer: I'm contributing the maximum to my Roth IRA, but I'm only contributing 5% of my salary to my 401(k).\nFinancial Advisor: Increasing your 401(k) contributions could be beneficial, especially if your employer offers matching contributions. Do they match your 401(k) contributions?",
 "Customer: Yes,

In [44]:
embedding  =  OpenAIEmbeddings()
conv_db =  Chroma.from_texts(texts=splits, embedding=embedding, persist_directory='./conv_db')

In [45]:
conv_db

In [46]:
def verify_key_points_with_semantic_search(conversation_chunks, key_points, vector_store):
    verification_results = {}
    
    for key_point in key_points.strip().split('\n'):
        key_point = key_point.strip()
        if key_point:
            # Perform semantic search
            result = vector_store.similarity_search(key_point, k=1)
            best_match = result[0].page_content if result else "Not found"
            verification_results[key_point] = best_match
            
    return verification_results

In [47]:
verification_results = verify_key_points_with_semantic_search(splits, key_points, conv_db)

In [48]:
print("\nVerification Results:")
for key_point, best_match in verification_results.items():
    print(f"Key Point: {key_point}\nBest Match: {best_match}\n")


Verification Results:
Key Point: Here is the key information from the conversation:
Best Match: Customer: Thank you for the advice. I feel more confident about my retirement planning now.
Financial Advisor: You’re welcome! If you have any other questions or need further assistance, feel free to reach out.

Key Point: - **Customer's Age**: 45 years old.
Best Match: Financial Advisor: Good morning! How can I assist you today?
Customer: Hi, I'm looking for some advice on retirement planning. I'm 45 years old and want to ensure I have enough saved up.
Financial Advisor: Sure, I'd be happy to help with that. Do you currently have any retirement accounts, such as a 401(k) or an IRA?
Customer: Yes, I have a 401(k) through my employer and a Roth IRA that I contribute to regularly.
Financial Advisor: Great. How much do you currently have saved in those accounts?

Key Point: - **Current Retirement Accounts**:
Best Match: Financial Advisor: Good morning! How can I assist you today?
Customer: Hi,